In [219]:
!pip install bio
from Bio.KEGG import REST
import json
import os

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [9]:
# Estrazione batteri
sngOrg = REST.kegg_get("br:br08401").read()
H_list = []
check = True
bact = False
for line in sngOrg.splitlines():
  if not check:
    break
  if line == "ABacterial infections":
    bact = True
    continue
  if bact:
    if line[0] == 'C':
      #print(line)
      H_list.append(line.split()[1])
  if bact and line[0] == 'A':
    check = False

In [18]:
#sngOrg = REST.kegg_list("ds").read()
#sngOrg = REST.kegg_get(["H02511", "H02512"]).read()
#sngOrg = REST.kegg_get("br:br08401").read()
#print(sngOrg)
#json_data = json.loads(sngOrg)
#H_list = []
#print(json_data['children'][0]['children'][0]['children'][0]['name'])
#print(len(json_data['children']))
#for line in sngOrg.splitlines():
#  if line[0] == 'C':
#    H_list.append(line.split()[1])
# H_list = []
#for line in sngOrg.splitlines():
#  code = line.split("\t")[0]
#  code_split = code.split(":")
#  if code_split[0] == "ds":
#    H_list.append(code_split[1])
print(H_list)
print(len(H_list))
no_paper = 0
drug_list = []
no_drug_list = []
all_get = ""
for h_id in H_list:
  tmp = REST.kegg_get(h_id).read()
  with open(f"kegg_get/{h_id}.txt", "w") as f:
    f.write(tmp)
  all_get = all_get + tmp
  all_get = all_get + "\n"
  if 'DRUG' in tmp:
    drug_list.append(h_id)
  else:
    no_drug_list.append(h_id)
  if not 'PMID' in tmp:
    #print(tmp)
    no_paper = no_paper + 1
print(drug_list)
print(no_drug_list)
#sngOrg = REST.kegg_get(["H00349"]).read()
#print('DRUG' in sngOrg)


['H00277', 'H00278', 'H00280', 'H01311', 'H01312', 'H00279', 'H01313', 'H01339', 'H00111', 'H00112', 'H00113', 'H00299', 'H00300', 'H00301', 'H01415', 'H00302', 'H00297', 'H00298', 'H00303', 'H01343', 'H01335', 'H01067', 'H01345', 'H01350', 'H01414', 'H00304', 'H01366', 'H01330', 'H00305', 'H00306', 'H01068', 'H01317', 'H01069', 'H00110', 'H00307', 'H00308', 'H01070', 'H00313', 'H01441', 'H00309', 'H01072', 'H01073', 'H00310', 'H00311', 'H00312', 'H01074', 'H00314', 'H00315', 'H01353', 'H01337', 'H01080', 'H00316', 'H00317', 'H00319', 'H01066', 'H01077', 'H01083', 'H01084', 'H01081', 'H01373', 'H00320', 'H01374', 'H01332', 'H00321', 'H01379', 'H00322', 'H02379', 'H00323', 'H01397', 'H00324', 'H01139', 'H01141', 'H01142', 'H01140', 'H01157', 'H00325', 'H01144', 'H00326', 'H00327', 'H01147', 'H01148', 'H01150', 'H01151', 'H01164', 'H01165', 'H01166', 'H01167', 'H01169', 'H00328', 'H00329', 'H01399', 'H00330', 'H01409', 'H00331', 'H01401', 'H01176', 'H01175', 'H00332', 'H01405', 'H00333',

In [12]:
print(len(drug_list))
print(len(no_drug_list))
print(no_paper)

62
103
1


In [148]:
#sngOrg = REST.kegg_get("gn:ype").read()

sngOrg = REST.kegg_get("ds:H00111").read()

print(sngOrg)

ENTRY       H00111                      Disease
NAME        Typhoid fever
DESCRIPTION Typhoid fever, or typhoid, caused by Salmonella enterica serovar Typhi is a systemic febrile disease and endemic disease in many areas of the world, mainly in developing countries. Typhoid is transmitted through ingestion of contaminated water or food and characterized by a variable incubation period ranging from three days to one month.
CATEGORY    Infectious disease
BRITE       Infectious diseases [BR:br08401]
             Bacterial infections
              Infections caused by enterobacteria
               H00111  Typhoid fever
            Human diseases in ICD-11 classification [BR:br08403]
             01 Certain infectious or parasitic diseases
              Gastroenteritis or colitis of infectious origin
               Bacterial intestinal infections
                1A07  Typhoid fever
                 H00111  Typhoid fever
PATHWAY     hsa05132  Salmonella infection
PATHOGEN    Salmonella enter

In [224]:
class paper:
   def __init__(self, pmid, authors, title, journal, doi):
    self.pmid = pmid
    self.authors = authors
    self.title = title
    self.journal = journal
    self.doi = doi

   def __repr__(self):
    return f"{self.title} ({self.pmid})\t{self.authors}\t{self.journal} ({self.doi})"
      
class drug:
  def __init__(self, name, id):
    self.name = name
    self.id = id

  def __repr__(self):
    return f"{self.name}, ({self.id})"

class bact:
  def __init__(self, name, id, description, category, sub, drugs, papers):
    self.name = name
    self.id = id
    self.drugs = drugs
    self.papers = papers
    self.description = description
    self.category = category
    self.sub = sub
  def __repr__(self):
    return f"{self.name} ({self.id})\n{self.drugs}\n ({self.papers})"

In [233]:
def parse(data):
  sub_bool = False
  sub_count = 0
  bact_sub = ""
  drug_bool = False
  paper_bool = False
  id_tmp = ""
  title_tmp = ""
  authors_tmp = ""
  journal_tmp = ""
  doi_tmp = ""
  doi_bool = False
  bact_name = ""
  bact_id = ""
  bact_cat = ""
  bact_des = ""
  drugs = []
  papers = []
  for line in data.splitlines():
    spl = line.split()
    if spl[0] == "NAME":
      bact_name = "".join(spl[i]+ " "  for i in range(1, len(spl)))
    if spl[0] == "ENTRY":
      bact_id = "".join(spl[i]+ " "  for i in range(1, len(spl))).split()[0]
    if spl[0] == "CATEGORY":
      bact_cat = "".join(spl[i]+ " "  for i in range(1, len(spl)))
    if spl[0] == "DESCRIPTION":
      bact_des = "".join(spl[i]+ " "  for i in range(1, len(spl)))

    if sub_bool == False and spl[0] == "BRITE":
      sub_bool = True
      sub_count += 1
    elif sub_bool == True and sub_count == 1:
      sub_count += 1
    elif sub_bool == True and sub_count == 2:
      sub_bool = False
      bact_sub = "".join(spl[i]+ " "  for i in range(0, len(spl)))

    if drug_bool == False and spl[0] == "DRUG":
      name = "".join(spl[i]+ " "  for i in range(1, len(spl)-1))
      drugs.append(drug(name, spl[-1].replace("[", "").replace("]", "")))
      drug_bool = True
    elif drug_bool == True and spl[0].upper() != spl[0]:
      name = "".join(spl[i]+ " "  for i in range(0, len(spl)))
      drugs.append(drug(name, spl[-1].replace("[", "").replace("]", "")))
    elif drug_bool == True and spl[0].upper() == spl[0]:
      drug_bool = False

    if paper_bool == False and spl[0] == "REFERENCE":
      if len(spl) > 1 and spl[1].split(":")[0] == "PMID":
        id_tmp = spl[1].split(":")[1].replace('\n',' ')
      paper_bool = True
    elif paper_bool == True and spl[0] == "AUTHORS":
      authors_tmp = "".join(spl[i]+ " "  for i in range(1, len(spl)))
    elif paper_bool == True and spl[0] == "TITLE":
      title_tmp = "".join(spl[i]+ " "  for i in range(1, len(spl)))
    elif paper_bool == True and spl[0] == "JOURNAL":
      journal_tmp = "".join(spl[i]+ " "  for i in range(1, len(spl)))
      doi_bool = True
    elif paper_bool == True and doi_bool == True and spl[0].upper() != spl[0]:
      doi_tmp = spl[0].split(":")[1].replace('\n',' ')
      doi_bool = False
      paper_bool = False
      papers.append(paper(id_tmp, authors_tmp, title_tmp, journal_tmp, doi_tmp))

  #print(drugs)
  #print(papers)

  bact_tmp = bact(bact_name, "ds:" + bact_id, bact_des, bact_cat, bact_sub, drugs, papers)
  #print(bact_tmp)
  return bact_tmp

In [225]:
bact_tmp = parse(sngOrg)

In [235]:
#print(bact)

bacts = []
for filename in os.listdir("kegg_get"):
  print(filename)
  with open("kegg_get/"+filename, "r") as f:
    bacts.append(parse(str(f.read())))


H01445.txt
H00340.txt
H01458.txt
H00313.txt
H01165.txt
H01426.txt
H01068.txt
H01440.txt
H00335.txt
H01515.txt
H00337.txt
H00328.txt
H01081.txt
H01317.txt
H00278.txt
H01072.txt
H01083.txt
H01049.txt
H00305.txt
H00353.txt
H00319.txt
H01050.txt
H01080.txt
H01021.txt
H01074.txt
H01353.txt
H01380.txt
H01077.txt
H00277.txt
H00301.txt
H00303.txt
H01454.txt
H00111.txt
H01407.txt
H00343.txt
H01064.txt
H01444.txt
H02029.txt
H01531.txt
H00342.txt
H00280.txt
H01164.txt
H00347.txt
H00388.txt
H01330.txt
H01339.txt
H00323.txt
H00316.txt
H00352.txt
H01167.txt
H00325.txt
H01517.txt
H01379.txt
H01410.txt
H01175.txt
H01332.txt
H00113.txt
H00314.txt
H00354.txt
H01343.txt
H01073.txt
H01148.txt
H00350.txt
H01157.txt
H00355.txt
H00326.txt
H00329.txt
H00315.txt
H00300.txt
H00333.txt
H01141.txt
H00332.txt
H01366.txt
H00351.txt
H01462.txt
H01142.txt
H01311.txt
H01373.txt
H01415.txt
H00339.txt
H01166.txt
H01314.txt
H01069.txt
H01472.txt
H00345.txt
H00309.txt
H01041.txt
H01374.txt
H00331.txt
H00341.txt
H01401.txt

In [236]:
print(bacts[0])

Acne vulgaris  (ds:H01445)
[Tetracycline hydrochloride , (DR:D02122), Doxycycline [DR:D00307] , (DR:D00307), Doxycycline calcium [DR:D03903] , (DR:D03903), Minocycline hydrochloride [DR:D00850] , (DR:D00850), Tazarotene [DR:D01132] , (DR:D01132), Tretinoin [DR:D00094] , (DR:D00094), Adapalene [DR:D01112] , (DR:D01112), Trifarotene [DR:D11225] , (DR:D11225), Clindamycin phosphate and tretinoin [DR:D11084] , (DR:D11084), Adapalene and benzoyl peroxide [DR:D10792] , (DR:D10792), Benzoyl peroxide [DR:D03093] , (DR:D03093), Clindamycin phosphate and benzoyl peroxide [DR:D10602] , (DR:D10602), Benzoyl peroxide and erythromycin [DR:D11646] , (DR:D11646), Clindamycin phosphate [DR:D01073] , (DR:D01073), Erythromycin [DR:D00140] , (DR:D00140), Sulfacetamide sodium [DR:D00871] , (DR:D00871), Dapsone [DR:D00592] , (DR:D00592), Clascoterone [DR:D11451] , (DR:D11451), Norgestimate and ethinyl estradiol [DR:D10590] , (DR:D10590), Sarecycline hydrochloride [DR:D10667] , (DR:D10667), Benzoyl peroxide 